## Descarga de valores para los ODS a nivel de municipio estimados con datos del Censo 2024 en Bolivia

[Fuente](https://fichas.ine.gob.bo/#/web/ods-cpv)

In [54]:
import requests
import pandas as pd
import json
from IPython.display import clear_output

Nos autenticamos en el servicio.

In [16]:
token = requests.post(
    "https://wfichas.ine.gob.bo/api/v1/seguridad/registrarSesion/", json={}
).json()["token"]

Descargamos todos los datos.

In [23]:
session = requests.Session()
session.headers.update({"Authorization": f"Bearer {token}"})

datos = []
mpio_count = 0
departamentos = session.get(
    "https://wfichas.ine.gob.bo/api/v1/dashboard/departamentos"
).json()["data"]
for departamento in departamentos:
    provincias = session.get(
        f"https://wfichas.ine.gob.bo/api/v1/dashboard/provincias/{departamento['cod_depto']}"
    ).json()["data"]
    for provincia in provincias:
        municipios = session.get(
            f"https://wfichas.ine.gob.bo/api/v1/dashboard/municipios/{provincia['con_prov']}"
        ).json()["data"]
        for municipio in municipios:
            mpio_count += 1
            ficha = session.get(
                f"https://wfichas.ine.gob.bo/api/v1/reportes/web/ods/01/0102/{municipio['con_mpio']}"
            ).json()["data"]
            unidad_administrativa = dict(
                departamento=departamento["depto"],
                id_departamento=departamento["cod_depto"],
                provincia=provincia["prov"],
                id_provincia=provincia["con_prov"],
                municipio=municipio["mpio"],
                id_municipio=municipio["con_mpio"],
            )
            clear_output(wait=True)
            print(f"{len(datos)} indicadores en {mpio_count} municipios.")
            for indicador in ficha["ods_01"]:
                keys = [
                    "tipo",
                    "nombre_indicador",
                    "ods",
                    "cdcm",
                    "pdes",
                    "a_2012_valor",
                    "a_2024_valor",
                ]
                valores_ods = {k: indicador[k] for k in keys}
                datos.append({**unidad_administrativa, **valores_ods})

5814 indicadores en 343 municipios.


Ésta es la tabla completa:

In [24]:
df = pd.DataFrame(datos)
df

,departamento,id_departamento,provincia,id_provincia,municipio,id_municipio,tipo,nombre_indicador,ods,cdcm,pdes,a_2012_valor,a_2024_valor
0,Chuquisaca,01,Azurduy,0102,Azurduy,010201,Complementario,Porcentaje de mujeres en edad fértil que su úl...,3-1-2,D-16,6-3-1,27.3,76.6
1,Chuquisaca,01,Azurduy,0102,Azurduy,010201,Complementario,Paridez media de mujeres de 15 a 19 años de edad.,3-7-2,B-10,6-3-1,0.4,0.4
2,Chuquisaca,01,Azurduy,0102,Azurduy,010201,Complementario,Población con Cobertura de Salud(2).,3-8-1,No aplica,6-3-1,0.0,90.6
3,Chuquisaca,01,Azurduy,0102,Azurduy,010201,Complementario,"Porcentaje de niñas, niños y adolescentes de 1...",4-1-1\n4-5-1,B-5,5-1-1,93.1,88.7
4,Chuquisaca,01,Azurduy,0102,Azurduy,010201,Complementario,Tasa de asistencia a un establecimiento educat...,4-2-2\n4-5-1,No aplica,5-1-1,25.7,61.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5826,Pando,09,Nicolás Suárez,0901,Porvenir,090102,Complementario,Porcentaje de población de 14 o más años de ed...,9-2-2,No aplica,1-4\n4-2,9.5,9.4
5827,Pando,09,Nicolás Suárez,0901,Porvenir,090102,Complementario,Porcentaje de hogares que tienen teléfono celu...,9-c-1,No aplica,1-3-4,0.0,83.6
5828,Pando,09,Nicolás Suárez,0901,Porvenir,090102,Complementario,Porcentaje de hogares que tienen internet.,9-c-1,No aplica,1-3-4,2.0,69.1
5829,Pando,09,Nicolás Suárez,0901,Porvenir,090102,Complementario,Porcentaje de niñas y niños menores de 5 años ...,16-9-1,No aplica,No aplica,89.1,98.0


Extraemos el conjunto de columnas que nos interesan más:

In [53]:
valores = df[
    [
        "departamento",
        "provincia",
        "municipio",
        "id_municipio",
        "nombre_indicador",
        "a_2012_valor",
        "a_2024_valor",
    ]
].rename(
    columns={
        "nombre_indicador": "indicador",
        "a_2012_valor": "2012",
        "a_2024_valor": "2024",
    }
).copy()
valores

,departamento,provincia,municipio,id_municipio,indicador,2012,2024
0,Chuquisaca,Azurduy,Azurduy,010201,Porcentaje de mujeres en edad fértil que su úl...,27.3,76.6
1,Chuquisaca,Azurduy,Azurduy,010201,Paridez media de mujeres de 15 a 19 años de edad.,0.4,0.4
2,Chuquisaca,Azurduy,Azurduy,010201,Población con Cobertura de Salud(2).,0.0,90.6
3,Chuquisaca,Azurduy,Azurduy,010201,"Porcentaje de niñas, niños y adolescentes de 1...",93.1,88.7
4,Chuquisaca,Azurduy,Azurduy,010201,Tasa de asistencia a un establecimiento educat...,25.7,61.5
...,...,...,...,...,...,...,...
5826,Pando,Nicolás Suárez,Porvenir,090102,Porcentaje de población de 14 o más años de ed...,9.5,9.4
5827,Pando,Nicolás Suárez,Porvenir,090102,Porcentaje de hogares que tienen teléfono celu...,0.0,83.6
5828,Pando,Nicolás Suárez,Porvenir,090102,Porcentaje de hogares que tienen internet.,2.0,69.1
5829,Pando,Nicolás Suárez,Porvenir,090102,Porcentaje de niñas y niños menores de 5 años ...,89.1,98.0


In [64]:
indicadores

{'Porcentaje de mujeres en edad fértil que su último parto fue atendido por personal de salud calificado(1).': {'ods': [{'meta': '3',
    'resultado': '1',
    'indicador': '2'}],
  'cdem': [{'capítulo': 'D', 'indicador': '16'}],
  'pdes': [{'eje': '3', 'meta': '1', 'resultado': '2'}]},
 'Paridez media de mujeres de 15 a 19 años de edad.': {'ods': [{'meta': '3',
    'resultado': '7',
    'indicador': '2'}],
  'cdem': [{'capítulo': 'B', 'indicador': '10'}],
  'pdes': [{'eje': '3', 'meta': '7', 'resultado': '2'}]},
 'Población con Cobertura de Salud(2).': {'ods': [{'meta': '3',
    'resultado': '8',
    'indicador': '1'}],
  'cdem': [{'capítulo': 'No aplica'}],
  'pdes': [{'eje': '3', 'meta': '8', 'resultado': '1'}]},
 'Porcentaje de niñas, niños y adolescentes de 12 a 17 años de edad que completaron al menos 6 años de escolaridad e índice de paridad.': {'ods': [{'meta': '4',
    'resultado': '1',
    'indicador': '1'},
   {'meta': '4', 'resultado': '5', 'indicador': '1'}],
  'cdem': [{'

Y las asociaciones de cada indicador con ODS, CdeM y PDES:

In [44]:
def documentar_indicador(indicador):
    def procesar_tipo_indicador(indicador_str, campos):
        resultado = []
        for i in indicador_str.split("\n"):
            resultado.append(
                {
                    name: value
                    for name, value in zip(
                        campos, i.split("-")
                    )
                }
            )
        return resultado

    ods = procesar_tipo_indicador(indicador["ods"], ["meta", "resultado", "indicador"])
    cdcm = procesar_tipo_indicador(indicador["cdcm"], ["capítulo", "indicador"])
    pdes = procesar_tipo_indicador(indicador["ods"], ["eje", "meta", "resultado"])

    return dict(
        ods=ods,
        cdem=cdcm,
        pdes=pdes
    )

In [49]:
indicadores = {
    nombre: documentar_indicador(entrada)
    for nombre, entrada in df[["nombre_indicador", "ods", "cdcm", "pdes"]]
    .drop_duplicates()
    .set_index("nombre_indicador")
    .iterrows()
}

Y guardamos todo.

In [55]:
valores.to_csv("datos.csv", index=False)
with open("indicadores.json", "w+") as f:
    json.dump(indicadores, f, ensure_ascii=False, indent=2)